# <strong>POSIX Permissions</strong>

POSIX (Portable Operating System Interface for UniX) permissions are important to understand because they're commonly used for configuring Unix environments among users. Companies that use Unix environments split the ownership of files among single users and groups. Permissions are an effective way of preventing certain users from reading, writing, or executing files and directories. By preventing specific users from being able to read, modify, and execute files, you are also creating a barrier of security in case users have malicious intent towards a company, or their account becomes compromised.

POSIX permissions can be viewed on any file by typing ```ls -l``` inside of a directory. A sample output of this command would be the following: 

<figure><center><img src="resources/posix/posix_example.png" style="width: 75%; height: 75%;"></img></center><em><figcaption>Credit: <a href="https://docs.oracle.com/cd/E19253-01/806-7612/files-26/index.html">Oracle Documentation</a></figcaption></em></figure>

For this lab, we are interested in changing the permissions, owner, and group. The way we read permissions is as follows:

<figure><center><img src="resources/posix/permissions_example.png" style="width: 75%; height: 75%;"></img></center><em><figcaption>Credit: <a href="https://www.semanticscholar.org/paper/POSIX-Access-Control-Lists-on-Linux-Gr%C3%BCnbacher/ab66cf18f024f9b1e56a14c9f233d753a1d5c878">Semantic Scholar</a></figcaption></em></figure>

There are three groups of users. First is the ```owner```, which is the user who's listed first in the POSIX output. The ```owner``` is the user who created the file or directory. The ```group``` is a set of people who are assigned to a tag/group within the Unix environment. Finally, ```other``` is everyone else that's not the ```owner``` nor in the ```group```.

There are three permissions that each type of user can acquire:
- ```r``` stands for ```read```. This permission allows the user to read the contents of a file or directory.
- ```w``` stands for ```write```. This permission allows the user to write to a directory or modify an existing file that has the ```write``` tag applied to it.
- ```x``` stands for ```execute```. This permission allows the user to run scripts or other executables within a directory. The ```x``` tag on a directory means that the user can navigate within the directory, but they may not be able to read the contents without the ```r``` tag applied to it.

<strong>This lab will contain four topics, and you will learn the following:</strong>

1. Symbolic POSIX Permissions
2. Octal POSIX Permissions
3. Special POSIX Permissions
4. Applying POSIX Permissions

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# These are pre-set so that the first three steps will work properly, as they all
# depend on each other. Remaining booleans will be set in the cells.
step1Complete = step2Complete = step3Complete = False

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/save.py intro"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/load.py intro"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "intro"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - umdsectc -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - umdsectc -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - umdsectc -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc detach xdc.umdsectc"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - umdsectc -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/umdsectc/notebooks/saves/umdsectc_intro.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                    output0_2.clear_output()
                    display(HTML("<span style='color: green;'>Your lab has been successfully loaded.</span>"))
            elif (result.returncode == 2):
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>The intro lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: Symbolic POSIX Permissions</strong>

For the first topic, you will learn how to set POSIX permissions symbolically. Setting permissions this way are more intuitive, as there are two different ways to set permissions: symbolically and octally. 

Permissions are set using the ```chmod``` command, which is short for "change mode". The "mode" of a file is another way of saying "permissions". When you change the permissions of a file that you do not own, you need to use ```sudo```. Otherwise, if you're the owner of a file, then ```sudo``` is not required. Here is an example of using the ```chmod``` command: ```sudo chmod u+x file```

A breakdown of the command:
- ```u``` can be either ```u``` for ```user/owner```, ```g``` for ```group```, and ```o``` for ```other```.
- ```+``` can be either ```+``` or ```-```, meaning you want to add or remove the following permission.
- ```x``` can be either ```r``` for reading, ```w``` for writing, or ```x``` for execution.
- ```file``` is the name of the file or directory that you would like to change the permissions of.

<u>Tips:</u> 
- You can use more than one letter when using ```chmod```. If you want to remove read/write permissions from ```owner``` and ```other```, you can use ```sudo chmod uo+rw file```</u>.
- If you do not want to treat ```+``` or ```-``` as a "switch on/off", you can also explicitly state what you want a user's permissions to be. For example: ```sudo chmod u=rwx file``` sets full permissions for the owner.

Using this information, answer the next few steps using the ```chmod``` command.

### Step 1: Set POSIX Permissions on ```q1.txt```.

Inside of the home directory is a folder called ```posix_practice/```, with a few files called ```q1.txt```, ```q2.txt```, and ```q3.txt```, inside of it. Currently, there are zero permissions on these files.

Give <strong>executable</strong> permissions to everyone for ```q1.txt```.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check the permissions.
def step_1():
    # Required to change boolean value.
    global step1Complete

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_1.py 1', shell=True)

    if (result.returncode == 1):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True
    
    elif (result.returncode == 2):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>q1.txt was not found. Check the directory and see if you may have deleted it.</span>"))
            step1Complete = False

    elif (result.returncode == 0):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Check your permissions and try again.</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 2: Set POSIX Permissions on ```q2.txt```.

Now, give <strong>write</strong> permissions to just the ```owner``` for ```q2.txt```.

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Required to change boolean value.
    global step2Complete

    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_1.py 2', shell=True)

    if (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True
    
    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>q2.txt was not found. Check the directory and see if you may have deleted it.</span>"))
            step2Complete = False

    elif (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Check your permissions and try again.</span>"))
            step2Complete = False

def check_step_2(b):
    step_2()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 3: Set POSIX Permissions on ```q3.txt```.

Finally, give <strong>read/write</strong> permissions to the ```owner```, <strong>executable</strong> permissions to the ```group```, and <strong>read/execute</strong> permissions to ```other``` for ```q3.txt```. <u>Do not add any other permissions besides the ones that were stated.</u>

In [6]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Required to change boolean value.
    global step3Complete

    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_1.py 3', shell=True)

    if (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True
    
    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>q3.txt was not found. Check the directory and see if you may have deleted it.</span>"))
            step2Complete = False

    elif (result.returncode == 0):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Check your permissions and try again.</span>"))
            step2Complete = False

def check_step_3(b):
    step_3()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 4: Short Answer

The next step is a short answer question. Therefore, it cannot be auto-graded.

<em>What does the ‘x’ bit allow you to do with directories?</em>

Type your answer in the answer box below, and click "Save Response" to save your response to your submission.

In [17]:
# Click the button below to check your work.
step4Complete = False

# Function to save the short answer.
def step_4():
    # Required to change boolean values.
    global step4Complete

    # Loading, in case saving is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Adding functionality here.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "echo \"' + userInput4.value + '\" > /home/.checker/responses/step_4_answer.txt"', shell=True)

    if (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
            step4Complete = True

    elif (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Your response was saved.</span>"))
            step4Complete = False
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

def check_step_4(b):
    step_4()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading4 = widgets.Output()
display(loading4)
with loading4:
    loading4.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area. We will need to assign output of process to the value of this input.
userInput4 = widgets.Textarea(
    placeholder='Type your response here',
    description='Response:',
    layout=widgets.Layout(width='75%', height='150px', margin='10px')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_4_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput4.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading4.clear_output()

# Creating the feedback output area.
output4 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(userInput4, button, output4)

Output()

Textarea(value='Hello!', description='Response:', layout=Layout(height='150px', margin='10px', width='75%'), p…

Button(description='Save Response', style=ButtonStyle())

Output()

## <strong>Topic 2: Octal POSIX Permissions</strong>

Now, you are ready to move onto the second topic, which is learning how to apply POSIX permissions using octal notation.

Symbolic notation is easy to use since you can explicitly state what user(s) have which permissions. However, if you are drastically changing the permissions, you may need to use multiple ```chmod``` commands, otherwise your ```chmod``` can begin to appear cluttered. <em>Why?</em> Take a look at this example:

```sudo chmod ug+rw,o-w+r,g+x``` will take the current permissions, add read/write privileges to the ```owner``` and ```group```, remove write permissions from ```other```, but give them read permissions, and give executable permissions to the group. However, the existing execution permissions for ```owner``` will remain unchanged, but with symbolic notation, you will not know what the current executable permissions are. ```owner``` may or may not have executable permissions, and we cannot tell based off of this command.

Octal notation is useful when you have an exact set of permissions that you want to use for a certain file or directory. The way that you read permissions in octal format is the following:

| Octal | Binary | Permissions |
|-------|--------|-------------|
| 0     | 000    | ---         |
| 1     | 001    | --x         |
| 2     | 010    | -w-         |
| 3     | 011    | -wx         |
| 4     | 100    | r--         |
| 5     | 101    | r-x         |
| 6     | 110    | rw-         |
| 7     | 111    | rwx         |

Notice that as a permission gets "toggled on", it displays a 1 in binary. Converting the number from binary to decimal is the octal representation of that user's permission. Since there are three users who can have permissions applied to them, there are three numbers that range from 0-7 in octal notation.

Now, to show an example. Whenever a file is made, it's set to ```-rw-r--r--``` by default. The octal representation of this permission is ```644```. 
<u>A breakdown:</u>
- ```owner```: ```rw-``` converts to ```110```, which is $2^2 + 2^1 + 0 = 4 + 2 + 0 = 6$
- ```group```: ```r--``` converts to ```100```, which is $2^2 + 0 + 0 = 4 + 0 + 0 = 4$
- ```other```: Same permissions as ```group```, which is $4$.
- Final octal permissions: ```644```

<center><strong>You may use a calculator below to calculate the octal notation of a given set of permissions.</strong></center>

In [8]:
# An interactive calculator for learning octal notation below.
from ipywidgets import Layout, Button, VBox, Label

# Function to update the octal and permission string.
def update_permissions(change):
    octal = 000
    permission_string = ""
    
    if owner_read.value:
        octal += 400
        permission_string += "r"
    else:
        permission_string += "-"
        
    if owner_write.value:
        octal += 200
        permission_string += "w"
    else:
        permission_string += "-"
        
    if owner_execute.value:
        octal += 100
        permission_string += "x"
    else:
        permission_string += "-"
    
    if group_read.value:
        octal += 40
        permission_string += "r"
    else:
        permission_string += "-"
        
    if group_write.value:
        octal += 20
        permission_string += "w"
    else:
        permission_string += "-"
        
    if group_execute.value:
        octal += 10
        permission_string += "x"
    else:
        permission_string += "-"
    
    if others_read.value:
        octal += 4
        permission_string += "r"
    else:
        permission_string += "-"
        
    if others_write.value:
        octal += 2
        permission_string += "w"
    else:
        permission_string += "-"
        
    if others_execute.value:
        octal += 1
        permission_string += "x"
    else:
        permission_string += "-"

    # Permissions done. Convert to string. Checking to see if it's only 1-2 digits long. If so, add padding to make it three digits.
    if (len(str(octal)) == 1):
        octal = "00" + str(octal)

    elif (len(str(octal)) == 2):
        octal = "0" + str(octal)

    elif (octal == 0):
        octal = str("000")
    
    octal_label.value = f"Octal Notation: {octal}"
    permission_label.value = f"Permission String: {permission_string}"

# Create widgets.
owner_read = widgets.Checkbox(value=False, description='Owner Read')
owner_write = widgets.Checkbox(value=False, description='Owner Write')
owner_execute = widgets.Checkbox(value=False, description='Owner Execute')

group_read = widgets.Checkbox(value=False, description='Group Read')
group_write = widgets.Checkbox(value=False, description='Group Write')
group_execute = widgets.Checkbox(value=False, description='Group Execute')

others_read = widgets.Checkbox(value=False, description='Others Read')
others_write = widgets.Checkbox(value=False, description='Others Write')
others_execute = widgets.Checkbox(value=False, description='Others Execute')

octal_label = widgets.Label(value="Octal Notation: 0")
permission_label = widgets.Label(value="Permissions: ---------")

# Create widgets
owner_read = widgets.Checkbox(value=False, description='Read')
owner_write = widgets.Checkbox(value=False, description='Write')
owner_execute = widgets.Checkbox(value=False, description='Execute')

group_read = widgets.Checkbox(value=False, description='Read')
group_write = widgets.Checkbox(value=False, description='Write')
group_execute = widgets.Checkbox(value=False, description='Execute')

others_read = widgets.Checkbox(value=False, description='Read')
others_write = widgets.Checkbox(value=False, description='Write')
others_execute = widgets.Checkbox(value=False, description='Execute')

octal_label = widgets.Label(value="Octal Notation: 000")
permission_label = widgets.Label(value="Permission String: ---------")

# Add observers to widgets
owner_read.observe(update_permissions, 'value')
owner_write.observe(update_permissions, 'value')
owner_execute.observe(update_permissions, 'value')

group_read.observe(update_permissions, 'value')
group_write.observe(update_permissions, 'value')
group_execute.observe(update_permissions, 'value')

others_read.observe(update_permissions, 'value')
others_write.observe(update_permissions, 'value')
others_execute.observe(update_permissions, 'value')

# Create columns for each user category
owner_column = widgets.VBox(
    [widgets.Label(value="Owner"), owner_read, owner_write, owner_execute],
    layout=widgets.Layout(overflow='hidden')
)
group_column = widgets.VBox(
    [widgets.Label(value="Group"), group_read, group_write, group_execute],
    layout=widgets.Layout(overflow='hidden')
)
others_column = widgets.VBox(
    [widgets.Label(value="Others"), others_read, others_write, others_execute],
    layout=widgets.Layout(overflow='hidden')
)

# Arrange the columns in a single row
permissions_table = widgets.HBox([owner_column, group_column, others_column])

# Display table and labels
display(permissions_table, octal_label, permission_label)

Label(value='Octal Notation: 000')

Label(value='Permission String: ---------')

### Step 5: Calculate Octal Permissions of ```q1.txt```

What is the octal representation of ```q1.txt``` that you previously set symbolically? Just use the three numbers for your response.

<u>You're encouraged to try answering this exercise without the calculator!</u> 

In [9]:
# Click the button below to check your work.
step5Complete = False

# Function to check if the student's answer was correct.
def step_5():
    # Required to change boolean values.
    global step1Complete, step2Complete, step3Complete, step5Complete

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, we need to see if the previous step(s) were completed.
    if (not step1Complete and not step2Complete and not step3Complete):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You must complete Steps 1-3 before attempting this question.</span>"))
            step5Complete = False

    # Next, check to make sure that the input is valid:
    elif (len(str(userInput5.value)) != 3 or not userInput5.value.isnumeric()):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Double check your octal permissions. It should be three numbers.</span>"))
            step5Complete = False

    # Input is valid.
    else:
        # Now, run the command.
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_2.py 5 ' + userInput5.value, shell=True, stdout=subprocess.DEVNULL)
        
        if (result.returncode == 1):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: green;'>Your octal value is correct!</span>"))
                step5Complete = True
    
        elif (result.returncode == 0):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>Double check your octal value.</span>"))
                step5Complete = False
    
        elif (result.returncode == 2):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>There was an error. Check to make sure that q*.txt files exist in ~/posix_practice.</span>"))
                step5Complete = False

def check_step_5(b):
    step_5()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading5 = widgets.Output()
display(loading5)
with loading5:
    loading5.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput5 = widgets.Text(
    placeholder='Type the octal value',
    description='Octal:',
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_5_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput5.value = result.stdout

# After the student's response was loaded, clear the output.
loading5.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(userInput5, button, output5)

Output()

Text(value='755', description='Octal:', placeholder='Type the octal value')

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 6: Calculate Octal Permissions of ```q2.txt```

What is the octal representation of ```q2.txt``` that you previously set symbolically? Just use the three numbers for your response.

<u>You're encouraged to try answering this exercise without the calculator!</u> 

In [10]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the student's answer was correct.
def step_6():
    # Required to change boolean values.
    global step1Complete, step2Complete, step3Complete, step6Complete

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, we need to see if the previous step(s) were completed.
    if (not step1Complete and not step2Complete and not step3Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You must complete Steps 1-3 before attempting this question.</span>"))
            step6Complete = False

    # Next, check to make sure that the input is valid:
    elif (len(str(userInput6.value)) != 3 or not userInput6.value.isnumeric()):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Double check your octal permissions. It should be three numbers.</span>"))
            step6Complete = False

    # Input is valid.
    else:
        # Now, run the command.
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_2.py 6 ' + userInput6.value, shell=True, stdout=subprocess.DEVNULL)
        
        if (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Your octal value is correct!</span>"))
                step6Complete = True
    
        elif (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>Double check your octal value.</span>"))
                step6Complete = False
    
        elif (result.returncode == 2):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>There was an error. Check to make sure that q*.txt files exist in ~/posix_practice.</span>"))
                step6Complete = False

def check_step_6(b):
    step_6()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput6 = widgets.Text(
    placeholder='Type the octal value',
    description='Octal:',
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput6.value = result.stdout

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(userInput6, button, output6)

Output()

Text(value='744', description='Octal:', placeholder='Type the octal value')

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 7: Calculate Octal Permissions of ```q3.txt```

What is the octal representation of ```q3.txt``` that you previously set symbolically? Just use the three numbers for your response.

<u>You're encouraged to try answering this exercise without the calculator!</u> 

In [11]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the student's answer was correct.
def step_7():
    # Required to change boolean values.
    global step1Complete, step2Complete, step3Complete, step7Complete

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, we need to see if the previous step(s) were completed.
    if (not step1Complete and not step2Complete and not step3Complete):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You must complete Steps 1-3 before attempting this question.</span>"))
            step7Complete = False

    # Next, check to make sure that the input is valid:
    elif (len(str(userInput7.value)) != 3 or not userInput7.value.isnumeric()):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>Double check your octal permissions. It should be three numbers.</span>"))
            step7Complete = False

    # Input is valid.
    else:
        # Now, run the command.
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_2.py 7 ' + userInput7.value, shell=True, stdout=subprocess.DEVNULL)
        
        if (result.returncode == 1):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: green;'>Your octal value is correct!</span>"))
                step7Complete = True
    
        elif (result.returncode == 0):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>Double check your octal value.</span>"))
                step7Complete = False
    
        elif (result.returncode == 2):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>There was an error. Check to make sure that q*.txt files exist in ~/posix_practice.</span>"))
                step7Complete = False

def check_step_7(b):
    step_7()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading7 = widgets.Output()
display(loading7)
with loading7:
    loading7.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput7 = widgets.Text(
    placeholder='Type the octal value',
    description='Octal:',
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_7_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput7.value = result.stdout

# After the student's response was loaded, clear the output.
loading7.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)

# Display the output.
display(userInput7, button, output7)

Output()

Text(value='615', description='Octal:', placeholder='Type the octal value')

Button(description='Check Permissions', style=ButtonStyle())

Output()

## <strong>Topic 3: Special POSIX Permissions</strong>

Next, you are going to experiment with special permissions. There are three types of special permissions that are used in Unix, with special cases to them. They are called the <strong>Set User ID (SUID)</strong> bit, the <strong>Set Group ID (SGID)</strong> bit, and the <strong>"sticky bit"</strong>.

The <strong>SUID</strong> bit on an executable file means that the file will run as the User ID of the file's owner as opposed to the User ID of the user executing the file. This is done to allow users to perform tasks that temporarily require the user to be someone else, such as changing passwords or restarting a service. Any program with the UID bit set must be carefully written so as to block all misuse. Innumerable vulnerabilities have stemmed from <strong>SUID</strong> root programs with security holes that allowed users to execute other commands as root.

The <strong>SGID</strong> bit on an executable file is like the <strong>SUID</strong> bit, except that the process gains the effective user of the file's group, not its owner or the user executing the file.

The <strong>"sticky bit"</strong> is a permission that's set on directories. It prevents anyone from being able to delete the directory, unless you're the owner itself. 

<em>Side Note: The "sticky bit" was used in the olden days to tell the kernel to keep an executable's image in memory so that it would not have to be reloaded from disk. This was commonly done with programs such as editors that were used regularly but had a significant load time. Modern systems use the "sticky bit" for other uses.</em>

There are different ways to apply these special permissions by using ```chmod```.

- Symbolic:
  - user + s(pecial): ```sudo chmod u+s file```
  - group + s(pecial): ```sudo chmod g+s file```
  - other + (s)t(icky): ```sudo chmod o+t dir```
- Octal: When applying the special permissions, your octal representation will change from 3-digits to 4-digits. The first digit is the special permission, and the last three digits are your standard permissions.
  - SUID: 4
  - SGID: 2
  - Sticky bit: 1
    - Example with applying the SGID: ```sudo chmod 2674 file```
   
Special permissions override the executable bit. When you apply the <strong>SUID</strong> bit, the permissions may look like the following: ```-rwsrw-rw-```. Since the special permission overrides the executable bit (in terms of display), an uppercase letter may be used to show that the special permission is applied <u>and</u> the file is executable by the owner. The example that was just shown indicates that the owner does not have executable permissions. To add executable permissions, the permissions will appear as: ```-rwSrw-rw-```.

### Step 8: Locating a Sticky Bit

Navigate to the top of the directory, which is the ```/``` directory. List the contents of it. Which directory contains the sticky bit? Type the folder name or the absolute path of it. (Fill in the blank)

In [12]:
# Click the button below to check your work.
step8Complete = False

# Function to check if the student's answer was correct.
def step_8():
    # Required to change boolean values.
    global step8Complete

    # Loading, in case the check is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Run the command.
    result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_3.py 8 ' + userInput8.value, shell=True, stdout=subprocess.DEVNULL)
    
    if (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Correct! You will need to use your solution to answer the next step.</span>"))
            step8Complete = True

    elif (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Double-check your answer.</span>"))
            step8Complete = False

    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>There was an error. Check to make sure that q*.txt files exist in ~/posix_practice.</span>"))
            step8Complete = False

def check_step_8(b):
    step_8()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading8 = widgets.Output()
display(loading8)
with loading8:
    loading8.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput8 = widgets.Text(
    placeholder='Type the directory here',
    description='Directory:',
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_8_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput8.value = result.stdout

# After the student's response was loaded, clear the output.
loading8.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)

# Display the output.
display(userInput8, button, output8)

Output()

Text(value='tmp', description='Directory:', placeholder='Type the directory here')

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 9: Explain Your Reasoning

The next step is a short answer question. Therefore, it cannot be auto-graded.

<em>Why is there a sticky bit specifically on the directory that you previously answered?</em>

Type your answer in the answer box below, and click "Save" to save your response to your submission. (Will add this later.)

In [13]:
# Click the button below to check your work.
step9Complete = False

# Function to save the short answer.
def step_9():
    # Required to change boolean values.
    global step9Complete

    # Loading, in case saving is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Adding functionality here.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "echo \"' + userInput9.value + '\" > /home/.checker/responses/step_9_answer.txt"', shell=True)

    if (result.returncode == 1):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
            step9Complete = True

    elif (result.returncode == 0):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: green;'>Your response was saved.</span>"))
            step9Complete = False
    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

def check_step_9(b):
    step_9()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading9 = widgets.Output()
display(loading9)
with loading9:
    loading9.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput9 = widgets.Textarea(
    placeholder='Type your response here',
    description='Response:',
    layout=widgets.Layout(width='75%', height='150px', margin='10px')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_9_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput9.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading9.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)

# Display the output.
display(userInput9, button, output9)

Output()

Textarea(value='asdf', description='Response:', layout=Layout(height='150px', margin='10px', width='75%'), pla…

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 10: Applying the Special Permissions

Inside of your home folder, there is a directory that's called ```special_permissions/```. This directory contains three files, ```suid.sh```, ```sgid.sh```, and ```sticky/```. Practice applying the SUID, SGID, and sticky bit to the appropriate files and directory.

In [14]:
# Click the button below to check your work.
step10Complete = False

# Function to check the permissions.
def step_10():
    # Required to change boolean value.
    global step10Complete

    with output10:
        output10.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_3.py 10 foo', shell=True)

    if (result.returncode == 1):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step10Complete = True
    
    elif (result.returncode == 2):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>q3.txt was not found. Check the directory and see if you may have deleted it.</span>"))
            step10Complete = False

    elif (result.returncode == 0):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>Check your permissions and try again.</span>"))
            step10Complete = False

def check_step_10(b):
    step_10()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output10 = widgets.Output()

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(button, output10)

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 11: Octal Notation with Special Permissions

Suppose a file exists where the ```owner``` has read permissions and Set-User ID, the ```group``` has write and executable permissions, and ```other``` has full permissions. What is the octal notation for this? (Fill in the blank)

In [15]:
# Click the button below to check your work.
step11Complete = False

# Function to check if the student's answer was correct.
def step_11():
    # Required to change boolean values.
    global step10Complete, step11Complete

    # Loading, in case the check is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, we need to see if the previous step(s) were completed.
    if (not step10Complete):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You must complete Step 10 before attempting this question.</span>"))
            step11Complete = False

    # Next, check to make sure that the input is valid:
    elif (len(str(userInput6.value)) != 4 or not userInput6.value.isnumeric()):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Double check your octal permissions. It should be four numbers.</span>"))
            step11Complete = False

    # Input is valid.
    else:
        # Now, run the command.
        result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_3.py 11 ' + userInput6.value, shell=True, stdout=subprocess.DEVNULL)
        
        if (result.returncode == 1):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: green;'>Your octal value is correct!</span>"))
                step11Complete = True
    
        elif (result.returncode == 0):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>Double check your octal value.</span>"))
                step11Complete = False
    
        elif (result.returncode == 2):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>There was an error. Check to make sure that q*.txt files exist in ~/posix_practice.</span>"))
                step11Complete = False

def check_step_11(b):
    step_11()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading11 = widgets.Output()
display(loading11)
with loading11:
    loading11.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput11 = widgets.Text(
    placeholder='Type the octal value',
    description='Octal:',
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/umdsectc/.ssh/merge_key umdsectc@posix "cat /home/.checker/responses/step_11_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput11.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading11.clear_output()

# Creating the button.
button = widgets.Button(description="Check Permissions")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(userInput11, button, output11)

Output()

Text(value='443', description='Octal:', placeholder='Type the octal value')

Button(description='Check Permissions', style=ButtonStyle())

Output()

### Step 12: Check your Understanding

True or False?

<em>1612 and 1613 set the same permissions.</em>

In [16]:
# Click the button below to check your work.
step12Complete = False

# Function to check if the student's answer was correct.
def step_12():
    # Required to change boolean values.
    global step12Complete

    # Checking the answer:
    result = subprocess.run('ssh -i /home/umdsectc/.ssh/merge_key umdsectc@posix /home/.checker/section_3.py 12 ' + truefalse1.value, shell=True, stdout=subprocess.DEVNULL)

    if (result.returncode == 1):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: green;'>Correct!</span>"))
            step12Complete = True

    elif (result.returncode == 0):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>Double-check your answer.</span>"))
            step12Complete = False
            
def check_step_12(b):
    step_12()

    """
    # Auto-save.
    if (not runAllSteps):
        trigger_save()
    """

# Creating the button.
button = widgets.Button(description="Check Answer")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Creating a text area.
truefalse1 = widgets.RadioButtons(
    options=['True', 'False'],
    description='Select your answer:'
)

# Display the output.
display(truefalse1, button, output12)

RadioButtons(description='Select your answer:', options=('True', 'False'), value='True')

Button(description='Check Answer', style=ButtonStyle())

Output()